In [1]:
import os
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/workspace/theislab/mubind/notebooks/batch

/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind/notebooks/batch


In [53]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations' # '../../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import numpy as np

In [54]:
df = mb.bindome.datasets.ProBound.ctcf(flank_length=0)
df = df.sort_values(1, ascending=False).reset_index(drop=True)
# data = df.head(1000)
# data = df.copy()

df.index = df['seq']
del df['seq']
df.index = df.index.astype(str).str[45:]

df = df[~df.index.duplicated(keep='first')]

n_sample = 5000
data = df.sample(min(df.shape[0], n_sample))

# data = mb.pp.sample_rounds(df, 2, 10000)

# remove as many nucleotides as posisble (faster training/convergence)


# data.index = data.index.astype(str).str[-15:]


In [55]:
df

,0,1
seq,,
CATGTAGACCGGCCATTGGACTATG,0,4
GCGTTGATGGTATCGTAAGTGCGTC,0,4
AGTGGCTGCTTGTCGGAATGGGCAG,0,4
GGCGTTACGCGTTGTGTAGACCTAA,0,4
CTCAGGTGGAATCTTTTTACTATTG,0,4
...,...,...
AGAGAGACAGGACTTGTGACTTTGT,1,0
AGAGGGCGTTCGAGGCTTGCTAGGC,1,0
GGAGTGCCTTTTGGATGAAAGATTG,1,0


In [56]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 5, 1

In [57]:
# TODO: skipped for now
n_rounds = 2

print('loading object (# entries)', data.shape[0])
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds, labels=[0, 1])
train = tdata.DataLoader(dataset=dataset,
                         # batch_size=256,
                         batch_size=625,
                         shuffle=False)

loading object (# entries) 5000


In [58]:
import warnings
warnings.filterwarnings('ignore')

In [59]:
# %lprun -f mb.tl.train_network mb.tl.train_network(model, train, device, next_optimiser, criterion, num_epochs=20, early_stopping=100, log_each=2, dirichlet_regularization=0, exp_max=40, verbose=0)

In [60]:
# %lprun -f mb.tl.train_iterative mb.tl.train_iterative(train, device, w=18, show_logo=False, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=[0.01, 0.01], weight_decay=[0.01, 0.001], ignore_kernel=ignore_kernel, num_epochs=2, early_stopping=100, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None)

In [61]:
pd.set_option('display.expand_frame_repr', False)


In [62]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 100
log_each = 10
n_kernels = 2
lr = [0.01] * n_kernels
kernels = [0, 2] + [20] * (n_kernels - 1)
wd = [0.01,] + [0.001] * (n_kernels - 1)
early_stopping = [5,] + [10] * (n_kernels)

opt_kernel_shift = [0, 0] + [1] * (n_kernels - 1)
opt_kernel_length = [0, 0] + [1] * (n_kernels - 1)
use_mono = True
use_dinuc = True
dinuc_mode = 'local'

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time

criterion = mb.tl.PoissonLoss()



In [63]:
model.device

device(type='cuda', index=0)

In [64]:
import itertools

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))


optimize_exp_barrier, optimize_kernel_rel, optimize_sym_weight = True, True, True
total_time = []
for use_cpu in [True, False]:

    t0 = time.time()

    print(optimize_exp_barrier, optimize_kernel_rel, optimize_sym_weight)
    model = mb.models.Multibind.make_model(train, n_kernels, criterion, kernels=kernels,
                                           device=device if not use_cpu else 'cpu',
                                           use_dinuc=use_dinuc, dinuc_mode=dinuc_mode,
                                           optimize_exp_barrier= optimize_exp_barrier,
                                           optimize_kernel_rel = optimize_kernel_rel,
                                           optimize_sym_weight = optimize_sym_weight) # .cuda()

    print(model.device)


    model, best_loss = model.optimize_iterative(train,
                                                show_logo=0,
                                                r2_per_epoch=True,
                                                opt_kernel_length=opt_kernel_length,
                                                opt_kernel_shift=opt_kernel_shift,
                                                dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                                # lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel,
                                                # skip_kernels={2, 3},
                                                n_epochs=n_epochs,
                                                early_stopping=early_stopping, # optimiser=torch.optim.LBFGS,
                                                log_each=log_each,
                                                stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
    print('##DONE....\n\n')



    print('total time: %.3f s' % ((time.time() - t0)))
    # res = []
    # model_by_k = {}

    # counts = mb.tl.predict(model, train)
    # print(counts)
    #
    # mb.pl.set_rcParams({'figure.figsize': [10, 6], 'figure.dpi': 65})
    # mb.pl.conv(model)
    #
    # mb.tl.scores(model, train, by='batch')
    # mb.pl.set_rcParams({'figure.figsize': [6, 6], 'figure.dpi': 85})
    # mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1')
    # mb.pl.activities(model)
    #
    # mb.pl.set_rcParams({'figure.figsize': [8, 4], 'figure.dpi': 85})
    # mb.pl.loss(model)

    total_time.append(model.total_time)
    # models.append([model, best_loss])
    # break


Using device: cuda:0
True True True
cpu
current kernels

Kernel to optimize 0

FREEZING KERNELS
optimizing feature type mono
setting grad status of kernel (mono, dinuc) at 0 to (1, 0)
setting grad status of kernel (mono, dinuc) at 1 to (0, 0)
setting grad status of kernel (mono, dinuc) at 2 to (0, 0)
kernels mask None
optimizer:  Adam 
criterion: PoissonLoss 
# epochs: 100 
early_stopping: 5
lr= 0.01, weight_decay= 0.001, dir weight= 0
Epoch: 11, Loss: 132.219751, R2: -1.10,  best epoch: 9,  secs per epoch: 0.424 s,  secs epoch*1k trials: 0.085s
Epoch: 21, Loss: 131.763746, R2: -0.94,  best epoch: 19,  secs per epoch: 0.369 s,  secs epoch*1k trials: 0.074s
Epoch: 31, Loss: 131.402328, R2: -0.50,  best epoch: 29,  secs per epoch: 0.352 s,  secs epoch*1k trials: 0.070s
Epoch: 41, Loss: 131.226582, R2: -0.08,  best epoch: 39,  secs per epoch: 0.378 s,  secs epoch*1k trials: 0.076s
Epoch: 51, Loss: 131.182749, R2: 0.03,  best epoch: 49,  secs per epoch: 0.378 s,  secs epoch*1k trials: 0.07

In [65]:
total_time[-2] / total_time[-1]

5.877846729961968

In [66]:
for m, best_loss in models:
    print(m.optimize_neg_weight, m.optimize_kernel_rel, m.optimize_sym_weight)
